In [1]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle competitions download -c anticipating-latency-in-cloud-environments
# !unzip anticipating-latency-in-cloud-environments.zip

  0% 0.00/1.53M [00:00<?, ?B/s]
100% 1.53M/1.53M [00:00<00:00, 88.4MB/s]
Archive:  anticipating-latency-in-cloud-environments.zip
  inflating: sample_submission.csv   
  inflating: testing_data.csv        
  inflating: training_data.csv       


In [2]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 3.6 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import lightgbm as lgb
from lightgbm import LGBMRegressor

In [2]:
df = pd.read_csv('/content/training_data.csv')
df

,ID,timestamp,cpu_type,cpu_limit,ram_limit,cpu_usage,ram_usage,num_req,conc_lvl,latency
0,2714,2021-11-04 23:15:38,Intel Xeon Gold 5317,2.0,70M,0.084450,46854144.0,100,71,7429000
1,38113,2021-11-04 23:23:27,AMD EPYC 72F3,2.0,70M,0.277893,55934976.0,300,21,2196000
2,33409,2021-11-11 13:32:59,Intel Xeon Platinum 8380,2.5,66M,0.351042,59109376.0,700,1,37000
3,10123,2022-03-19 02:35:05,Intel Xeon Platinum 8356H,0.8,768M,0.026230,43659264.0,100,41,3946000
4,12815,2022-03-17 06:30:29,AMD EPYC 72F3,0.4,512M,0.108346,51048448.0,800,51,5084000
...,...,...,...,...,...,...,...,...,...,...
41129,54343,2022-03-18 02:05:09,Intel Xeon Silver 4309Y,0.7,896M,0.169227,50458624.0,900,11,1039000
41130,38158,2022-03-11 14:24:38,Intel Core i9-11900K,1.5,640M,0.147585,50913280.0,300,1,53000
41131,860,2022-03-16 16:28:38,Intel Xeon Platinum 8352M,1.8,640M,0.134895,51859456.0,900,31,3145000
41132,15795,2022-03-17 03:18:29,Intel Xeon Platinum 8368,0.7,768M,0.203862,45862912.0,800,21,2001000


In [3]:
df.describe()

,ID,cpu_limit,cpu_usage,ram_usage,num_req,conc_lvl,latency
count,41134.000000,41134.000000,41134.000000,4.113400e+04,41134.000000,41134.000000,4.113400e+04
mean,29357.337434,1.445007,0.205489,5.377951e+07,532.958137,37.228424,6.483698e+06
std,16909.992230,1.077288,0.141493,7.771771e+06,286.221513,27.515045,1.492553e+07
min,0.000000,0.100000,0.000000,0.000000e+00,100.000000,1.000000,2.500000e+04
25%,14796.250000,0.500000,0.109756,4.941005e+07,300.000000,11.000000,1.175250e+06
50%,29311.500000,1.300000,0.167758,5.224448e+07,500.000000,31.000000,3.756000e+06
75%,43993.750000,2.000000,0.312084,5.872845e+07,800.000000,51.000000,6.022000e+06
max,58762.000000,4.000000,3.998097,1.371832e+08,1000.000000,101.000000,2.667150e+08


In [4]:
def convert_ram_limit(df):
  df['ram_limit'] = df['ram_limit'].str[:-1].astype(int)

In [5]:
def encode(df):
  convert_ram_limit(df)
  df['year'] = pd.to_datetime(df['timestamp']).dt.year
  df['month'] = pd.to_datetime(df['timestamp']).dt.month
  df["month"] = (pd.to_datetime(df['timestamp']).dt.year - 2021) * 12 + (pd.to_datetime(df['timestamp']).dt.month)
  df['day'] = pd.to_datetime(df['timestamp']).dt.day
  # df['month'] = df['month'] + (df['day'] /30)
  df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
  df.drop('timestamp', axis=1, inplace=True)
  df.drop('year', axis=1, inplace=True)
  df.drop('day', axis=1, inplace=True)
  df.drop('cpu_type', axis=1, inplace=True)
  df['ram_usage'] = df['ram_usage'] / 1000000
  # df['ram_usage'] = df['ram_limit'] - df['ram_usage']
  df['num_req'] = df['num_req'] / 1000
  return df
encode(df)
# df = pd.get_dummies(df, columns=['cpu_type'], dtype=int)
df

,ID,cpu_limit,ram_limit,cpu_usage,ram_usage,num_req,conc_lvl,latency,month,hour
0,2714,2.0,70,0.084450,46.854144,0.1,71,7429000,11,23
1,38113,2.0,70,0.277893,55.934976,0.3,21,2196000,11,23
2,33409,2.5,66,0.351042,59.109376,0.7,1,37000,11,13
3,10123,0.8,768,0.026230,43.659264,0.1,41,3946000,15,2
4,12815,0.4,512,0.108346,51.048448,0.8,51,5084000,15,6
...,...,...,...,...,...,...,...,...,...,...
41129,54343,0.7,896,0.169227,50.458624,0.9,11,1039000,15,2
41130,38158,1.5,640,0.147585,50.913280,0.3,1,53000,15,14
41131,860,1.8,640,0.134895,51.859456,0.9,31,3145000,15,16
41132,15795,0.7,768,0.203862,45.862912,0.8,21,2001000,15,3


In [6]:
df['month'].unique()

array([11, 15, 10], dtype=int32)

In [7]:
X = df.drop(['ID', 'latency'], axis=1)
y = df['latency']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
def evaluate(model):
  y_pred = model.predict(X_test)
  return r2_score(y_pred, y_test)

In [59]:
LR = LinearRegression()
LR.fit(X_train, y_train)

LinearRegression()

In [24]:
evaluate(LR)

-1.8633738758787906

In [11]:
RF = RandomForestRegressor(n_estimators=100)
RF.fit(X_train, y_train)

RandomForestRegressor()

In [12]:
evaluate(RF)

0.921754998966788

In [13]:
params_xgb = {
    'learning_rate': 0.1,
    'n_estimators': 100,
    'max_depth': 8,
}

# Initialize and train the model
XGB = XGBRegressor()
XGB.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [14]:
evaluate(XGB)

0.9125587999107194

In [15]:
params_cb = {
    'iterations': 1200,
    'learning_rate': 0.1,
    'depth': 8,
    'l2_leaf_reg': 3,
    'bootstrap_type': 'Bayesian',
    'bagging_temperature': 1,
    'rsm': 1,
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE',
    'logging_level': 'Silent',
}

# Initialize and train the model
CB = CatBoostRegressor()
CB.fit(X_train, y_train)

Learning rate set to 0.071107
0:	learn: 14227910.6050491	total: 58.5ms	remaining: 58.4s
1:	learn: 13569883.5673158	total: 64.9ms	remaining: 32.4s
2:	learn: 12911842.5684021	total: 70.8ms	remaining: 23.5s
3:	learn: 12323095.6499207	total: 76.2ms	remaining: 19s
4:	learn: 11768943.3065552	total: 81.9ms	remaining: 16.3s
5:	learn: 11247854.6168156	total: 87.2ms	remaining: 14.5s
6:	learn: 10784826.8053579	total: 92.8ms	remaining: 13.2s
7:	learn: 10366634.4772560	total: 100ms	remaining: 12.4s
8:	learn: 9972314.0322755	total: 108ms	remaining: 11.9s
9:	learn: 9643073.6372670	total: 113ms	remaining: 11.2s
10:	learn: 9305534.0756919	total: 119ms	remaining: 10.7s
11:	learn: 9024878.5499027	total: 125ms	remaining: 10.3s
12:	learn: 8767413.7170447	total: 131ms	remaining: 9.92s
13:	learn: 8508470.0306705	total: 136ms	remaining: 9.6s
14:	learn: 8294870.4475931	total: 142ms	remaining: 9.33s
15:	learn: 8069546.4343905	total: 148ms	remaining: 9.09s
16:	learn: 7886124.3641784	total: 153ms	remaining: 8.87s

In [16]:
evaluate(CB)

0.9240811180448315

In [17]:
gbm = GradientBoostingRegressor(n_estimators=500, learning_rate=0.1)
gbm.fit(X_train, y_train)

GradientBoostingRegressor(n_estimators=500)

In [18]:
evaluate(gbm)

0.8568261437624752

In [19]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Set LightGBM parameters
params = {
    'objective': 'regression',  # 'regression' for regression tasks
    'metric': 'rmse',  # 'rmse' for Root Mean Squared Error
    'num_leaves': 80,  # Maximum tree leaves for base learners
    'learning_rate': 0.1,  # Learning rate (shrinkage)
    'feature_fraction': 0.8,  # Feature fraction (randomly select a subset of features)
    'bagging_fraction': 0.8,  # Bagging fraction (randomly select a subset of data)
    'bagging_freq': 5,  # Frequency for bagging
    'verbose': 0  # Controls the level of LightGBM's verbosity
}

# Train the LightGBM model
num_round = 300  # Number of boosting rounds
lgb_model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

In [20]:
evaluate(lgb_model)

0.9395299468042884

In [21]:
lgb_reg = LGBMRegressor(
    objective='regression',
    num_leaves=120,
    learning_rate=0.1,
    n_estimators=100,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=5
)
lgb_reg.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005196 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 625
[LightGBM] [Info] Number of data points in the train set: 32907, number of used features: 8
[LightGBM] [Inf

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, feature_fraction=0.8,
              num_leaves=120, objective='regression')

In [22]:
evaluate(lgb_reg)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


0.9317850276196269

In [437]:
voting_reg = VotingRegressor(
    estimators=[('XGB', XGB), ('catboost', CB), ('LGB', lgb_reg), ('RF', RF)]
)
voting_reg.fit(X_train, y_train)

Learning rate set to 0.071107
0:	learn: 14107481.0366071	total: 20.6ms	remaining: 20.6s
1:	learn: 13288176.3688976	total: 35.6ms	remaining: 17.7s
2:	learn: 12500820.8926067	total: 51.5ms	remaining: 17.1s
3:	learn: 11784243.0401722	total: 65.6ms	remaining: 16.3s
4:	learn: 11135829.9234586	total: 80.1ms	remaining: 15.9s
5:	learn: 10511764.0381232	total: 94.8ms	remaining: 15.7s
6:	learn: 9966271.9928912	total: 110ms	remaining: 15.7s
7:	learn: 9442968.1836684	total: 126ms	remaining: 15.6s
8:	learn: 8974838.4761312	total: 140ms	remaining: 15.4s
9:	learn: 8512469.4615495	total: 162ms	remaining: 16s
10:	learn: 8105567.3731987	total: 181ms	remaining: 16.3s
11:	learn: 7724341.1030523	total: 203ms	remaining: 16.7s
12:	learn: 7357027.3866478	total: 222ms	remaining: 16.8s
13:	learn: 7032299.4504310	total: 244ms	remaining: 17.2s
14:	learn: 6743322.3060178	total: 265ms	remaining: 17.4s
15:	learn: 6467559.2524052	total: 279ms	remaining: 17.2s
16:	learn: 6214307.6448138	total: 295ms	remaining: 17.1s
1

VotingRegressor(estimators=[('XGB',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...)),
                            ('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7baf793397b0>),
                            ('LGB',
                             LGBMRegressor(bagging_fraction=0.8, bagging_freq=5,
                                           feature_fraction=0.8, num_leaves=120,
                                           objective='regression')),
                            ('RF', RandomForestRegressor())])

In [438]:
evaluate(voting_reg)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


0.9729164971040377

## Submission

In [23]:
voting_reg = VotingRegressor(
    estimators=[('XGB', XGB), ('catboost', CB), ('LGB', lgb_reg), ('RF', RF)]
)
voting_reg.fit(X, y)

Learning rate set to 0.073659
0:	learn: 14137110.4044179	total: 8.96ms	remaining: 8.96s
1:	learn: 13425155.2230109	total: 16.6ms	remaining: 8.28s
2:	learn: 12759228.9082499	total: 23.2ms	remaining: 7.7s
3:	learn: 12154736.3679882	total: 29.6ms	remaining: 7.37s
4:	learn: 11599338.7048510	total: 36.2ms	remaining: 7.21s
5:	learn: 11082979.9859657	total: 42.7ms	remaining: 7.07s
6:	learn: 10632042.7105309	total: 49.2ms	remaining: 6.97s
7:	learn: 10211553.9919137	total: 55.5ms	remaining: 6.88s
8:	learn: 9828493.9437577	total: 62ms	remaining: 6.83s
9:	learn: 9502400.7970530	total: 68.6ms	remaining: 6.79s
10:	learn: 9169515.3099082	total: 75.2ms	remaining: 6.76s
11:	learn: 8861656.7178581	total: 81.7ms	remaining: 6.73s
12:	learn: 8605307.1032239	total: 88.7ms	remaining: 6.73s
13:	learn: 8351454.6359056	total: 95.4ms	remaining: 6.72s
14:	learn: 8119976.0952961	total: 105ms	remaining: 6.9s
15:	learn: 7909369.1232372	total: 111ms	remaining: 6.85s
16:	learn: 7719196.1960005	total: 118ms	remaining:

VotingRegressor(estimators=[('XGB',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...)),
                            ('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7b06e82c7e20>),
                            ('LGB',
                             LGBMRegressor(bagging_fraction=0.8, bagging_freq=5,
                                           feature_fraction=0.8, num_leaves=120,
                                           objective='regression')),
                            ('RF', RandomForestRegressor())])

In [24]:
X_submission_test = pd.read_csv('/content/testing_data.csv')
X_submission = X_submission_test.drop('ID', axis=1)
X_submission = encode(X_submission)
# X_submission = pd.get_dummies(X_submission, columns=['cpu_type'], dtype=int)

In [25]:
prediction = voting_reg.predict(X_submission)
prediction = prediction.astype(int)
submission = pd.DataFrame({'ID': X_submission_test['ID'], 'latency': prediction})
submission.to_csv('submission.csv', index=False)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
